Berikut adalah dokumentasi dalam format Markdown untuk proyek klasifikasi sampah menggunakan ResNet, diikuti oleh implementasi kode.

---

# Klasifikasi Sampah Menggunakan ResNet dengan TensorFlow

## Pendahuluan
Klasifikasi sampah menggunakan deep learning adalah metode yang dapat membantu otomatisasi dalam pemilahan dan pengelolaan limbah secara efisien. Model berbasis arsitektur Residual Network (ResNet) mampu mengenali jenis sampah dengan menggunakan dataset gambar yang dilabeli, yang berpotensi memperbaiki proses daur ulang otomatis. ResNet sering digunakan dalam pengenalan gambar karena kemampuannya mengatasi masalah vanishing gradient melalui blok residual yang mempertahankan informasi penting dari lapisan sebelumnya [(He et al., 2016)](https://consensus.app/papers/resnet-deep-learning-he/9c586abb395db4c2a24d1a4e18388f23/).

## Lingkungan dan Pengaturan Awal
- **Library yang Digunakan**: TensorFlow digunakan sebagai framework utama untuk deep learning, bersama pustaka `pandas`, `numpy`, dan `matplotlib` untuk manipulasi data dan visualisasi. `wandb` digunakan untuk pelacakan metrik model dan logging eksperimen, yang membantu dalam memantau performa model dan stabilitas selama pelatihan [(Biewald, 2020)](https://consensus.app/papers/weights-biases-open-source-biewald/e1289d2819c3a3d8cb70d44f6a5587b5/).
- **Dataset TrashNet**: Dataset gambar dari berbagai jenis sampah, seperti kertas, plastik, dan kaca, digunakan untuk melatih model. TrashNet menyediakan data yang beragam dan relevan untuk kebutuhan klasifikasi sampah otomatis [(Yang & Thung, 2017)](https://consensus.app/papers/classification-trash-using-deep-learning-yang-thung/e65627e5d9095e9232ff2f193fef10a7/).

## Arsitektur Model ResNet
ResNet menggunakan struktur blok residual untuk memungkinkan sinyal informasi melewati jaringan dengan minimal gangguan, mengatasi masalah vanishing gradient yang umum pada jaringan dalam [(He et al., 2016)](https://consensus.app/papers/resnet-deep-learning-he/9c586abb395db4c2a24d1a4e18388f23/).

- **Blok Residual**: Blok residual memungkinkan shortcut connections yang membawa input asli dari satu lapisan ke lapisan berikutnya, menjaga integritas informasi dari lapisan awal [(Szegedy et al., 2015)](https://consensus.app/papers/going-deeper-convolutions-szegedy/1a4e1e4b0f45310d711eaac8ae945a17/).
- **Arsitektur Model**: Model dimulai dengan lapisan konvolusi besar, diikuti dengan pooling dan beberapa blok residual. Global Average Pooling digunakan di akhir jaringan, yang memberikan stabilitas saat menangani data gambar dalam jumlah besar dan bervariasi.

## Pembagian Data
Untuk melatih model dengan data yang seimbang, dataset dibagi menjadi tiga set: pelatihan, validasi, dan pengujian. Pembagian manual memastikan data terdistribusi secara merata, meminimalkan risiko bias [(James et al., 2013)](https://consensus.app/papers/introduction-statistical-learning-james-et-al/09d17878513a84a3e95c748c8b6e5e93/).

## Preprocessing dan Augmentasi Data
- **Normalisasi dan Deteksi Blur**: Gambar normalisasi dan deteksi blur diterapkan pada setiap gambar agar kualitas gambar menjadi lebih baik sebelum dimasukkan ke model. Normalisasi dapat meningkatkan akurasi dan stabilitas model [(Krizhevsky et al., 2012)](https://consensus.app/papers/image-net-classification-deep-convolutional-krizhevsky-sutskever-hinton/2e5c8ac2f47d97cf9fc9b515f9aa2b24/).
- **Augmentasi Data**: Augmentasi seperti flip horizontal, rotasi, dan penyesuaian kecerahan digunakan untuk memperkaya data gambar. Teknik ini terbukti meningkatkan kemampuan generalisasi model pada data yang belum pernah dilihat sebelumnya [(Shorten & Khoshgoftaar, 2019)](https://consensus.app/papers/survey-image-data-augmentation-deep-shorten-khoshgoftaar/c2c52f2dd34891f128cc27ff18bc9e48/).

## Pelatihan Model
Model dilatih pada data pelatihan dan dievaluasi pada data validasi untuk menguji akurasi dan stabilitas. Hanya model yang mencapai akurasi dan stabilitas tertentu yang disimpan, untuk menghindari model overfitting atau underfitting [(Goodfellow et al., 2016)](https://consensus.app/papers/deep-learning-goodfellow-bengio-courville/90a54b117d70e0309978f7d0af87a4ef/).

## Evaluasi dan Prediksi Model
Evaluasi model menggunakan data pengujian untuk menghitung loss dan akurasi, memberikan gambaran yang lebih baik tentang performa model pada data baru. Akurasi tinggi menunjukkan kemampuan model dalam mengklasifikasikan sampah dengan baik [(Russakovsky et al., 2015)](https://consensus.app/papers/imagenet-large-scale-visual-recognition-challenge-russakovsky-deng/66e0a771282f27a07b05f98f315d2be3/).

---

In [1]:
import datasets # type: ignore
import tensorflow as tf # type: ignore
import numpy as np
import random
from PIL import Image
from tensorflow.keras.preprocessing import image  # type: ignore
from tensorflow.keras.applications import ResNet50  # type: ignore
from tensorflow.keras import layers, models  # type: ignore
import wandb # type: ignore
import tensorflow as tf # type: ignore
import sys

# Memeriksa dan mengatur GPU
gpus = tf.config.list_physical_devices('GPU')
if not gpus:
    print("Tidak ada GPU yang terdeteksi. Program ini memerlukan GPU untuk dijalankan.")
    sys.exit(1)  # Menghentikan program dengan kode status 1

try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    print("Pengaturan memori GPU berhasil.")
except Exception as e:
    print(f"Kesalahan saat mengatur memori GPU: {e}")
    sys.exit(1)  # Menghentikan program jika terjadi kesalahan saat mengatur GPU


2024-11-08 11:45:04.768910: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-08 11:45:04.768985: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-08 11:45:04.771002: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-08 11:45:04.788455: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-08 11:45:06.811112: W tensorflow/compiler/tf2

Pengaturan memori GPU berhasil.


2024-11-08 11:45:10.178407: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-08 11:45:10.274334: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-08 11:45:10.274723: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:

# Fungsi untuk memproses dan melakukan augmentasi pada gambar
def preprocess_and_augment_images(dataset, num_images=0, target_size=(224, 224)):
    train_images = []
    train_labels = []

    for i in range(min(num_images, len(dataset['train']))):
        try:
            img = dataset['train'][i]['image'].convert('RGB').resize(target_size)
            label = dataset['train'][i]['label']
            img_array = np.array(img) / 255.0
            
            # Deteksi blur dan augmentasi gambar
            if np.std(img_array) < 0.05:
                img_array *= 2  # Tingkatkan kontras
            
            augment_type = random.choice(['flip', 'rotate', 'brightness'])
            if augment_type == 'flip':
                img_array = np.fliplr(img_array)
            elif augment_type == 'rotate':
                img_array = np.rot90(img_array)
            elif augment_type == 'brightness':
                factor = random.uniform(0.5, 1.5)
                img_array = np.clip(img_array * factor, 0, 1)

            train_images.append(img_array)
            train_labels.append(label)

        except Exception as e:
            print(f"Error processing image {i + 1}: {e}")

    return np.array(train_images), np.array(train_labels)

# Fungsi untuk membuat model ResNet50
def create_resnet50(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=base_model.input, outputs=outputs)

    for layer in base_model.layers:
        layer.trainable = False  # Membekukan lapisan dasar

    return model

# Fungsi untuk melatih dan menyimpan model dengan kondisi tertentu
def train_and_save_model(train_images, train_labels, val_images, val_labels, num_classes, epochs=10):
    model = create_resnet50(input_shape=(224, 224, 3), num_classes=num_classes)

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(train_images, train_labels,
                        epochs=epochs,
                        validation_data=(val_images, val_labels))

    # Menyimpan model jika memenuhi syarat akurasi
    if history.history['val_accuracy'][-1] >= 0.80:
        model.save('simpan_resnet50_model.h5')
        print("Model telah disimpan.")
    
    return model

# Fungsi untuk mengevaluasi model
def evaluate_model(model, test_images, test_labels):
    loss, accuracy = model.evaluate(test_images, test_labels)
    print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Fungsi utama untuk menjalankan program
def main():
    wandb.login(key="967fafd5558eeeff3ce5681cf55c71633438428d")  # Ganti dengan kunci API Anda
    # Memuat dataset
    ds = datasets.load_dataset("garythung/trashnet")

    num_images = 5024
    train_images, train_labels = preprocess_and_augment_images(ds, num_images=num_images)

    # Memisahkan data menjadi set pelatihan dan validasi
    split_index = int(0.8 * len(train_images))  # 80% untuk pelatihan
    val_images, val_labels = train_images[split_index:], train_labels[split_index:]  # 20% untuk validasi
    train_images, train_labels = train_images[:split_index], train_labels[:split_index]  # 80% untuk pelat ihan

    # Menentukan jumlah kelas
    num_classes = len(set(train_labels))

    # Mengupdate parameter config berdasarkan dataset
    config = {
        "learning_rate": 0.001,
        "epochs": 10,
        "batch_size": 64,
        "architecture": "ResNet50",
        "dataset": ds.info.description,  # Deskripsi dataset
        "num_classes": num_classes  # Jumlah kelas
    }
    
    wandb.init(project='my-tf-trash', config=config)

    model = train_and_save_model(train_images, train_labels, val_images, val_labels, num_classes) 

if __name__ == "__main__":
    main()

wandb: Currently logged in as: kerjta (kerjta-kkkk). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
